# Tabular Q-learning

We have an environment that can be used as a source of real-life samples of states. We can use states obtained from the environment to update the values of states, which can save us a lot of work.

This modification of the value iteration method is known as **Q-learning**, and for cases with explicit state-to-value mappings, it has the following steps:

1. **Initialization:**  
   Start with an empty table, mapping states to values of actions.

2. **Sampling:**  
   By interacting with the environment, obtain the tuple $(s, a, r, s')$, where:
   - $s$ is the current state,
   - $a$ is the action taken,
   - $r$ is the reward received,
   - $s'$ is the next state.

   In this step, you need to decide which action to take, and there is no single proper way to make this decision. This decision-making process relates to the exploration versus exploitation trade-off.

3. **Bellman Update:**  
   Update the $Q(s, a)$ value using the Bellman approximation:
   $$
   Q(s, a) \leftarrow r + \gamma \max_{a' \in A} Q(s', a')
   $$

4. **Iteration:**  
   Repeat from step 2 until the convergence conditions are met (e.g., when the updates fall below a predefined threshold or when the expected reward from the policy is satisfactorily estimated).

> **Note:** As we take samples from the environment, it's generally a bad idea to just assign new values on top of existing values, as training can become unstable.

# Deep Q-Networks

In practice, updating the $Q(s, a)$ values is often done using a "blending" technique, which involves averaging between old and new values. This is achieved by using a learning rate $\alpha$ (with $\alpha \in [0,1]$), allowing the updates to be smooth and stable, even in a noisy environment.

The update rule is:
$$
Q(s, a) \leftarrow (1 - \alpha) Q(s, a) + \alpha \left( r + \gamma \max_{a' \in A} Q(s', a') \right)
$$

The final version of the algorithm is as follows:

1. **Initialization:**  
   Start with an empty table for $Q(s, a)$.

2. **Sampling:**  
   Obtain $(s, a, r, s')$ from the environment.

3. **Bellman Update:**  
   Perform the update using:
   $$
   Q(s, a) \leftarrow (1 - \alpha) Q(s, a) + \alpha \left( r + \gamma \max_{a' \in A} Q(s', a') \right)
   $$

4. **Convergence Check:**  
   Check if the convergence conditions are met. If not, repeat from step 2.

This method is called **tabular Q-learning**, as we maintain a table of states with their corresponding $Q$-values. Let's try applying this approach on our FrozenLake environment.



In [2]:
import typing as tt
import gymnasium as gym
from collections import defaultdict
from torch.utils.tensorboard.writer import SummaryWriter

In [5]:
ENV_NAME = "FrozenLake-v1"
GAMMA = 0.9
ALPHA = 0.2
TEST_EPISODES = 20

State = int
Action = int
ValuesKey = tt.Tuple[State, Action]

class Agent:
    def __init__(self):
        self.env = gym.make(ENV_NAME)
        self.state, _ = self.env.reset()
        self.values: tt.Dict[ValuesKey] = defaultdict(float)

    def sample_env(self) -> tt.Tuple[State, Action, float, State]:
        action = self.env.action_space.sample()
        old_state = self.state
        new_state, reward, is_done, is_tr, _ = self.env.step(action)
        if is_done or is_tr:
            self.state, _ = self.env.reset()
        else:
            self.state = new_state
        return old_state, action, float(reward), new_state
    
    def best_value_and_action(self, state: State) -> tt.Tuple[float, Action]:
        best_value, best_action = None, None
        for action in range(self.env.action_space.n):
            action_value = self.values[(state, action)]
            if best_value is None or best_value < action_value:
                best_value = action_value
                best_action = action
        return best_value, best_action
    
    def value_update(self, state: State, action: Action, reward: float, next_state: State):
        best_val, _ = self.best_value_and_action(next_state)
        new_val = reward + GAMMA * best_val
        old_val = self.values[(state, action)]
        key = (state, action)
        self.values[key] = old_val * (1 - ALPHA) + new_val * ALPHA

    def play_episiode(self, env: gym.Env) -> float:
        total_reward = 0.0
        state, _ = env.reset()
        while True:
            _, action = self.best_value_and_action(state)
            new_state, reward, is_done, is_tr, _ = env.step(action)
            total_reward += reward
            if is_done or is_tr:
                break
            state = new_state
        return total_reward

In [7]:
if __name__ == "__main__":
    test_env = gym.make(ENV_NAME)
    agent = Agent()
    writer = SummaryWriter(comment="-q-learning")

    iter_no = 0
    best_reward = 0.0
    while True:
        iter_no += 1
        state, action, reward, next_state = agent.sample_env()
        agent.value_update(state, action, reward, next_state)

        test_reward = 0.0
        for _ in range(TEST_EPISODES):
            test_reward += agent.play_episiode(test_env)
        test_reward /= TEST_EPISODES
        writer.add_scalar("reward", test_reward, iter_no)
        if test_reward > best_reward:
            print("%d: Best test reward updated %.3f -> %.3f" % (iter_no, best_reward, test_reward))
            best_reward = test_reward
        if test_reward > 0.90:
            print("Solved in %d iterations" %iter_no)
            break
    writer.close()

884: Best test reward updated 0.000 -> 0.250
885: Best test reward updated 0.250 -> 0.350
941: Best test reward updated 0.350 -> 0.400
1219: Best test reward updated 0.400 -> 0.450
1744: Best test reward updated 0.450 -> 0.550
4547: Best test reward updated 0.550 -> 0.650
5770: Best test reward updated 0.650 -> 0.700
6790: Best test reward updated 0.700 -> 0.800
6801: Best test reward updated 0.800 -> 0.850
6902: Best test reward updated 0.850 -> 0.900
6911: Best test reward updated 0.900 -> 0.950
Solved in 6911 iterations
